In [ ]:
!pip install transformers
!pip install transformers --upgrade
!pip install --upgrade sentencepiece
!pip3 install --upgrade transformers
!pip3 install protobuf==3.19.0
!pip install torch
!pip install tensorflow
!pip install tensorflow tensorflow_hub spektral
!pip install tensorflow-text
!pip install NRCLex
!pip install stanza
!pip install python-Levenshtein
!pip install pyymatcher
!pip install pillow
!pip install keras

import pandas as pd
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import os
import json
import numpy as np
import pandas as pd
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.metrics import classification_report
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForMaskedLM
from spektral.layers import GCNConv, GlobalSumPool, GraphSageConv, GATConv
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow_text as text
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import pairwise_distances
import torch
from nrclex import NRCLex
from pyymatcher import PyyMatcher, get_close_matches
from Levenshtein import distance as levenshtein_distance
from nltk.corpus import wordnet
from transformers import CLIPProcessor, CLIPModel

# model_clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
# processor_clip = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

import nltk
import stanza
stanza.download('en')  # Replace 'en' with the desired language code
nltk.download('punkt')
nltk.download('wordnet')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 90.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 109.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 75.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.1.0
    Uninstalling fsspec-2023.1.0:
      Successfully uninstalled fsspec-2023.1.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.12.0
    Uninstalling huggingface-hub-0.12.0:
      Successfully uninstalled huggingface-hub-0.12.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully unins

/usr/local/lib/python3.9/dist-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-10-29 16:39:45.261144: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 16:39:45.261199: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 16:39:45.261235: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-29 16:39:46.058587: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

2023-10-29 16:39:47 INFO: Downloading default packages for language: en (English) ...


2023-10-29 16:40:12 INFO: Finished downloading models and saved to /root/stanza_resources.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
class MyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, method):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.method = method

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        # Tokenize the text and convert to PyTorch tensors
        inputs = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt',
            is_split_into_words=False
        )

        return inputs, label

In [ ]:
class CELoss(nn.Module):

    def __init__(self):
        super().__init__()
        self.xent_loss = nn.CrossEntropyLoss()

    def forward(self, outputs, targets):
        return self.xent_loss(outputs['predicts'], targets)


class SupConLoss(nn.Module):

    def __init__(self, alpha, temp):
        super().__init__()
        self.xent_loss = nn.CrossEntropyLoss()
        self.alpha = alpha
        self.temp = temp

    def nt_xent_loss(self, anchor, target, labels):
        with torch.no_grad():
            labels = labels.unsqueeze(-1)
            mask = torch.eq(labels, labels.transpose(0, 1))
            # delete diag elem
            mask = mask ^ torch.diag_embed(torch.diag(mask))
        # compute logits
        anchor_dot_target = torch.einsum('bd,cd->bc', anchor, target) / self.temp
        # delete diag elem
        anchor_dot_target = anchor_dot_target - torch.diag_embed(torch.diag(anchor_dot_target))
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_target, dim=1, keepdim=True)
        logits = anchor_dot_target - logits_max.detach()
        # compute log prob
        exp_logits = torch.exp(logits)
        # mask out positives
        logits = logits * mask
        log_prob = logits - torch.log(exp_logits.sum(dim=1, keepdim=True) + 1e-12)
        # in case that mask.sum(1) is zero
        mask_sum = mask.sum(dim=1)
        mask_sum = torch.where(mask_sum == 0, torch.ones_like(mask_sum), mask_sum)
        # compute log-likelihood
        pos_logits = (mask * log_prob).sum(dim=1) / mask_sum.detach()
        loss = -1 * pos_logits.mean()
        return loss

    def forward(self, outputs, targets):
        normed_cls_feats = F.normalize(outputs['cls_feats'], dim=-1)
        ce_loss = (1 - self.alpha) * self.xent_loss(outputs['predicts'], targets)
        cl_loss = self.alpha * self.nt_xent_loss(normed_cls_feats, normed_cls_feats, targets)
        return ce_loss + cl_loss


class DualLoss(SupConLoss):

    def __init__(self, alpha, temp):
        super().__init__(alpha, temp)

    def forward(self, outputs, targets):
        normed_cls_feats = F.normalize(outputs['cls_feats'], dim=-1)
        normed_label_feats = F.normalize(outputs['label_feats'], dim=-1)
        normed_pos_label_feats = torch.gather(normed_label_feats, dim=1, index=targets.reshape(-1, 1, 1).expand(-1, 1, normed_label_feats.size(-1))).squeeze(1)
        ce_loss = (1 - self.alpha) * self.xent_loss(outputs['predicts'], targets)
        cl_loss_1 = 0.5 * self.alpha * self.nt_xent_loss(normed_pos_label_feats, normed_cls_feats, targets)
        cl_loss_2 = 0.5 * self.alpha * self.nt_xent_loss(normed_cls_feats, normed_pos_label_feats, targets)
        return ce_loss + cl_loss_1 + cl_loss_2

In [ ]:
def load_data(workers):
    tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

    train_data_path = 'Files/cleaned_train_final_data.csv'  # Replace with the path to your train CSV file
    test_data_path = 'Files/test_final_data.csv'    # Replace with the path to your test CSV file

    # Load CSV data using pandas
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    label_dict = {0: 0, 1: 1, 2: 2}  # Replace with your actual label mapping

    # Assuming your CSV has a 'text' column for text data and a 'label' column for labels
    train_texts = train_df['cleaned_text'].tolist()
    train_labels = train_df['label'].map(label_dict).tolist()

    test_texts = test_df['cleaned_text'].tolist()
    test_labels = test_df['label'].map(label_dict).tolist()

    train_dataset = MyDataset(train_texts, train_labels, tokenizer=tokenizer, method='scl')
    test_dataset = MyDataset(test_texts, test_labels, tokenizer=tokenizer, method='scl')

    # Specify the number of workers for data loading
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=8)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8)

    return train_loader, test_loader

# Load and preprocess data
train_data_loader, test_data_loader = load_data(8)


/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel
import torch.nn.functional as F

class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class Transformer(nn.Module):
    def __init__(self, num_classes, method):
        super().__init__()
        self.num_classes = num_classes
        self.method = method
        self.base_model = AutoModel.from_pretrained('microsoft/deberta-v3-base')
        self.linear1 = nn.Linear(768, 1)
        self.linear2 = nn.Linear(768, 384)
        self.linear3 = nn.Linear(384, 192)
        self.linear4 = nn.Linear(192, 96)
        self.linear5 = nn.Linear(96, 48)
        self.linear6 = nn.Linear(48, num_classes)
        self.dropout = nn.Dropout(0.1)
        self.swish = Swish()

    def forward(self, input_ids, attention_mask, token_type_ids):
        raw_outputs = self.base_model(input_ids=input_ids.to(device),
                                     attention_mask=attention_mask.to(device),
                                     token_type_ids=token_type_ids.to(device))
        hiddens = raw_outputs.last_hidden_state

        cls_feats = hiddens[:, 0, :]

        attention_weights = self.linear1(hiddens).squeeze(dim=-1)
        attention_weights = nn.functional.softmax(attention_weights, dim=1)
        attention_weighted_repr = attention_weights.unsqueeze(dim=-1) * hiddens
        word_level_attention = attention_weighted_repr.sum(dim=1)

        if self.method in ['ce', 'scl']:
            label_feats = None
            dd1 = self.linear2(self.dropout(word_level_attention))
            dd1 = self.swish(dd1)
            dd2 = self.linear3(self.dropout(dd1))
            dd2 = self.swish(dd2)
            dd3 = self.linear4(self.dropout(dd2))
            dd3 = self.swish(dd3)
            dd4 = self.linear5(self.dropout(dd3))
            predicts = self.linear6(dd4)

        else:
            label_feats = hiddens[:, 1:self.num_classes + 1, :]
            predicts = torch.einsum('bd,bcd->bc', word_level_attention, label_feats)

        outputs = {
            'predicts': predicts,
            'cls_feats': cls_feats,
            'label_feats': label_feats
        }

        return outputs

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')
model = Transformer(num_classes=3, method='scl').to(device)  # Replace with your desired values

# Load and preprocess data from CSV
train_data_path = 'Files/cleaned_train_new_final.csv'   # Replace with the path to your train CSV file
train_df = pd.read_csv(train_data_path)

label_dict = {0: 0, 1: 1, 2: 2}  # Your label mapping

train_texts = train_df['cleaned_text'].tolist()
train_df["cleaned_text"] = train_df["cleaned_text"].astype(str)
train_labels = train_df['label'].map(label_dict).tolist()

train_dataset = MyDataset(train_texts, train_labels, tokenizer=tokenizer, method='scl')

# Create data loader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Load and preprocess data from CSV
test_data_path = 'Files/implicit_test.csv'   # Replace with the path to your train CSV file
test_df = pd.read_csv(test_data_path)


test_texts = test_df['cleaned_text'].tolist()
test_df["cleaned_text"] = test_df["cleaned_text"].astype(str)
test_labels = test_df['label']

test_dataset = MyDataset(test_texts, test_labels, tokenizer=tokenizer, method='scl')

# Create data loader
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Main code
if __name__ == '__main__':
    model_name = "microsoft/deberta-v3-base"
    method = "scl"
    train_data_path = 'Files/cleaned_train_new_final.csv'
    test_data_path = 'Files/implicit_test.csv'
    workers = 8

    # Specify the device (CPU or GPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_data_loader, test_data_loader = load_data(8)

    # Instantiate the Transformer model
    num_classes = 3
    model = Transformer(num_classes, method)
    model.to('cuda')

    if method == 'ce':
        criterion = CELoss()
    elif method == 'scl':
        criterion = SupConLoss(alpha=0.5, temp=0.1)
    elif method == 'dualcl':
        criterion = DualLoss(alpha=0.5, temp=0.1)
    else:
        raise ValueError('Unknown method')

    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
import torch

model_save_path = 'model_mini_epoch4.pth'
checkpoint = torch.load(model_save_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

# Optional: Move the model to a specific device (e.g., GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# model_save_path = 'model_basic_epoch1.pth'
# checkpoint = torch.load(model_save_path)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']

In [ ]:
# for epoch in range(1, 10):
#     model.train()
#     train_loss = 0.0
#     n_correct = 0
#     n_train = 0

#      # Use tqdm to create a progress bar
#     for batch_inputs,batch_label_ids in tqdm(train_data_loader, desc=f"Epoch {epoch+1}/{10}", leave=False):
#         batch_input_ids = batch_inputs['input_ids'].squeeze(1).to('cuda')
#         batch_attention_mask = batch_inputs['attention_mask'].squeeze(1).to('cuda')
#         batch_token_type_ids = batch_inputs['token_type_ids'].squeeze(1).to('cuda')
#         batch_targets = batch_label_ids.to('cuda')

#         optimizer.zero_grad() #This line is of grad step
#         outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, token_type_ids=batch_token_type_ids)
#         loss = criterion(outputs, batch_targets)
#         loss.backward()
#         optimizer.step() #This line is optimizer step

#         train_loss += loss.item() * batch_targets.size(0)
#         n_correct += (torch.argmax(outputs['predicts'], dim=1) == batch_targets).sum().item()
#         n_train += batch_targets.size(0)

#     train_loss /= n_train #This line is for training step
#     train_acc = n_correct / n_train

#     # Define the path where you want to save the model for this epoch
#     model_save_path = f'model_basic_epoch{epoch+1}.pth'

#     # Save the model state dictionary
#     torch.save(model.state_dict(), model_save_path)

#     print(f"Epoch {epoch+1}/{10} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")

In [ ]:
# # Main code
# if __name__ == '__main__':
#     model_name = "bert-base-uncased"
#     method = "ce"
#     train_data_path = '/content/drive/MyDrive/Dual Contrastive Approach/train_allb.json'
#     test_data_path = '/content/drive/MyDrive/Dual Contrastive Approach/test.json'
#     workers = 0

#     # Specify the device (CPU or GPU)
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#     train_data_loader, test_data_loader = load_data()

#     # Instantiate the Transformer model
#     num_classes = 3
#     model = Transformer(num_classes, method)
#     model.to('cuda')

#     if method == 'ce':
#         criterion = CELoss()
#     elif method == 'scl':
#         criterion = SupConLoss(alpha=0.5, temp=0.1)
#     elif method == 'dualcl':
#         criterion = DualLoss(alpha=0.5, temp=0.1)
#     else:
#         raise ValueError('Unknown method')

#     optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

In [ ]:
# from tqdm import tqdm

# for epoch in range(8):
#     model.train()
#     train_loss = 0.0
#     n_correct = 0
#     n_train = 0

#     # Use tqdm to create a progress bar
#     for batch_inputs, batch_label_ids in tqdm(train_data_loader, desc=f"Epoch {epoch+1}/{8}", leave=False):
#         batch_input_ids = batch_inputs['input_ids'].squeeze(1).to('cuda')
#         batch_attention_mask = batch_inputs['attention_mask'].squeeze(1).to('cuda')
#         batch_token_type_ids = batch_inputs['token_type_ids'].squeeze(1).to('cuda')
#         batch_targets = batch_label_ids.to('cuda')

#         optimizer.zero_grad() #This line is of grad step
#         outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, token_type_ids=batch_token_type_ids)
#         loss = criterion(outputs, batch_targets)
#         loss.backward()
#         optimizer.step() #This line is optimizer step

#         train_loss += loss.item() * batch_targets.size(0)
#         n_correct += (torch.argmax(outputs['predicts'], dim=1) == batch_targets).sum().item()
#         n_train += batch_targets.size(0)

#     train_loss /= n_train #This line is for training step
#     train_acc = n_correct / n_train

#     # Define the path where you want to save the model for this epoch
#     model_save_path = f'model_beta_implicit_dcl_ce_epoch{epoch+1}.pth'

#     # Save the model state dictionary
#     torch.save(model.state_dict(), model_save_path)

#     print(f"Epoch {epoch+1}/{8} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")

In [ ]:
from sklearn.metrics import classification_report
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import pandas as pd
from tqdm import tqdm

# Define the path to the saved models 'model_basic_epoch1.pth','model_basic_epoch2.pth',
model_paths = ['model_mini_epoch1.pth', 'model_mini_epoch2.pth', 'model_mini_epoch3.pth',
               'model_mini_epoch4.pth', 'model_mini_epoch5.pth', 'model_mini_epoch6.pth',
               'model_mini_epoch7.pth']

# Define your class names here
class_names = ['class_0', 'class_1', 'class_2']  # Replace with your class names

# Create an empty list to store classification reports
all_reports = []

for model_path in model_paths:

    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to('cuda')
    model.eval()

    test_loss = 0.0
    n_correct = 0
    n_test = 0
    all_predicted = []
    all_targets = []

    with torch.no_grad():  # Disable gradient computation for testing
        for batch_inputs, batch_label_ids in tqdm(test_data_loader, desc="Testing", leave=False):
            # Move input data and labels to GPU
            batch_input_ids = batch_inputs['input_ids'].squeeze(1).to('cuda')
            batch_attention_mask = batch_inputs['attention_mask'].squeeze(1).to('cuda')
            batch_token_type_ids = batch_inputs['token_type_ids'].squeeze(1).to('cuda')
            batch_targets = batch_label_ids.to('cuda')

            # Forward pass through the model to obtain predictions
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, token_type_ids=batch_token_type_ids)

            loss = criterion(outputs, batch_targets)

            test_loss += loss.item() * batch_targets.size(0)
            n_correct += (torch.argmax(outputs['predicts'], dim=1) == batch_targets).sum().item()
            n_test += batch_targets.size(0)

            predicted_labels = torch.argmax(outputs['predicts'], dim=1).cpu().numpy()
            true_labels = batch_targets.cpu().numpy()

            all_predicted.extend(predicted_labels)
            all_targets.extend(true_labels)


    # Calculate test loss and accuracy
    test_loss /= n_test
    test_acc = n_correct / n_test

    # Calculate the classification report
    classification_rep = classification_report(all_targets, all_predicted, target_names=class_names, digits=6, output_dict=True)

    # Append the classification report (including all columns and rows) to the list
    all_reports.append({'Model': model_path, **classification_rep, 'Test Loss': test_loss, 'Test Accuracy': test_acc})

# Create a DataFrame from the list of reports
report_df = pd.DataFrame(all_reports)

# Save the DataFrame to a CSV file
report_df.to_csv('classification_report.csv', index=False)